In [138]:
import pandas as pd

***

# Customer Table

In [139]:
df_customer = pd.read_csv('.\Customer.csv')

In [140]:
df_customer  # 10,000 rows

,CustomerID,Billing City,Billing Country,Gender,Date of Birth,Creation Date,First Order Date,Email Opt In
0,4110,Wetzlar,Germany,Female,1982-04-01,2017-12-07,2018-01-01,NaN
1,4115,NaN,NaN,NaN,NaN,2018-03-13,2018-05-21,NaN
2,4116,NaN,NaN,NaN,1998-08-06,2018-05-01,NaN,NaN
3,4117,East Sussex,United Kingdom,NaN,NaN,2014-11-28,2014-11-28,NaN
4,4118,Castro Valley,United States,NaN,NaN,2016-11-30,2016-11-30,True
...,...,...,...,...,...,...,...,...
9995,2038,Union City,United States,Male,2015-07-15,2014-04-04,NaN,True
9996,2039,NaN,NaN,NaN,NaN,2019-02-24,NaN,NaN
9997,2043,Birmingham,United Kingdom,Male,NaN,2015-08-22,2015-08-22,NaN
9998,2044,Miami,United States,Female,1998-08-15,2017-07-17,2017-07-19,True


In [141]:
df_customer.CustomerID.value_counts() ## all of the 10,000 customer ID's are unique.

2049    1
8865    1
6806    1
4759    1
8857    1
       ..
9526    1
5432    1
7481    1
1338    1
2047    1
Name: CustomerID, Length: 10000, dtype: int64

In [142]:
df_customer.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CustomerID        10000 non-null  int64 
 1   Billing City      6512 non-null   object
 2   Billing Country   6721 non-null   object
 3   Gender            2703 non-null   object
 4   Date of Birth     2620 non-null   object
 5   Creation Date     10000 non-null  object
 6   First Order Date  7091 non-null   object
 7   Email Opt In      2423 non-null   object
dtypes: int64(1), object(7)
memory usage: 625.1+ KB


### ^ Lots of incomplete data, only customer ID and creation date full


In [143]:
df_customer.describe(include = ['object', 'int'])    

,Billing City,Billing Country,Gender,Date of Birth,Creation Date,First Order Date,Email Opt In
count,6512,6721,2703,2620,10000,7091,2423
unique,3861,79,4,2163,1689,1460,3
top,London,United States,Female,1995-07-25,2018-11-19,2018-11-19,True
freq,99,2458,1698,5,422,461,1631


## ^ From the general description of the columns, want to investigate Gender, Email Opt In, and the ranges of dated columns

## Gender column:

In [144]:
df_customer.Gender.value_counts(dropna = False)

NaN       7297
Female    1698
Male       973
W           22
M           10
Name: Gender, dtype: int64

- ### Most customers have no gender assigned (7297)
- ### Rest are 4 types rather than 2

- ### Assuming the W and M are 'Woman' and 'Man', they should be corrected to Female and Male for consistency

## Email Opt In column

In [145]:
df_customer['Email Opt In'].value_counts(dropna = False)

NaN      7577
True     1631
False     735
X          57
Name: Email Opt In, dtype: int64

### ^ We probably expect this to be binary, however there is missing data. The X's should be 'false' for consistency

## Date Columns

### Customer account creation date

In [146]:
df_customer['Creation Date'] = pd.to_datetime(df_customer['Creation Date']) 

print("Most recent customer account: ", df_customer['Creation Date'].max())
print("Oldest customer account: ", df_customer['Creation Date'].min())

Most recent customer account:  2019-05-19 00:00:00
Oldest customer account:  2012-01-13 00:00:00


In [147]:
print(df_customer['Creation Date'].value_counts(bins=8, sort = False, dropna = False))

(2012-01-10 07:36:28.799999999, 2012-12-13 09:00:00]      13
(2012-12-13 09:00:00, 2013-11-13 18:00:00]                96
(2013-11-13 18:00:00, 2014-10-15 03:00:00]               208
(2014-10-15 03:00:00, 2015-09-15 12:00:00]               622
(2015-09-15 12:00:00, 2016-08-15 21:00:00]               714
(2016-08-15 21:00:00, 2017-07-17 06:00:00]              1509
(2017-07-17 06:00:00, 2018-06-17 15:00:00]              2558
(2018-06-17 15:00:00, 2019-05-19]                       4280
Name: Creation Date, dtype: int64


### ^ Potentially a skewed dataset due to error in the dates accounts were created, or simply the website became more popular over time. Can investigate if there is an error, such as potential mass duplication in creation date during the later end of the date range

In [148]:
df_customer.duplicated(subset='Creation Date').value_counts() 

True     8311
False    1689
dtype: int64

In [149]:
df_customer['Creation Date'].value_counts()

2018-11-19    422
2017-11-20    134
2018-07-02    121
2015-06-01    114
2018-11-20     98
             ... 
2015-07-02      1
2015-10-23      1
2013-08-18      1
2016-08-10      1
2017-08-27      1
Name: Creation Date, Length: 1689, dtype: int64

### ^ Most accounts were created on 2018-11-19. It could be an error, or simply due to the website's popularity.

### Customer Date of Births

In [150]:
print(df_customer['Date of Birth'].isna().value_counts())

True     7380
False    2620
Name: Date of Birth, dtype: int64


### ^ Majority of customers have no DOB attached to their account, or are not in the same format to be classified as such

In [151]:
df_customer['Date of Birth'] = pd.to_datetime(df_customer['Date of Birth']) 

print("Youngest Customer: ", df_customer['Date of Birth'].max())
print("Oldest Customer: ", df_customer['Date of Birth'].min())

Youngest Customer:  2019-05-03 00:00:00
Oldest Customer:  1946-11-16 00:00:00


### ^ The availabile range of DOBs, a fairly typical spread for a gymwear brand ↓

In [152]:
print(df_customer['Date of Birth'].value_counts(bins=8, sort = False, dropna = False))

(1946-10-20 12:48:57.599999999, 1955-12-07 06:00:00]       3
(1955-12-07 06:00:00, 1964-12-27 12:00:00]                19
(1964-12-27 12:00:00, 1974-01-17 18:00:00]                84
(1974-01-17 18:00:00, 1983-02-08]                        120
(1983-02-08, 1992-02-29 06:00:00]                        527
(1992-02-29 06:00:00, 2001-03-21 12:00:00]              1687
(2001-03-21 12:00:00, 2010-04-11 18:00:00]               124
(2010-04-11 18:00:00, 2019-05-03]                         56
Name: Date of Birth, dtype: int64


In [209]:
print((df_customer['Date of Birth'] >= '2014-05-03 00:00:00').value_counts())

False    9951
True       49
Name: Date of Birth, dtype: int64


### ^ Looking at customer Date of births, and knowing the most recent order date from the Order table (further down↓) is Dec 2019, it is unlikely that 49 customers are younger than 5, or 56 being 9 or younger.  

### ^ Additionally unlikely some customers are over the age of 65 however this is more realistic than those under 10 purchasing gymwear

 ***

## Order Table

In [154]:
df_order = pd.read_csv('.\Order.csv')

In [155]:
df_order ## 9,896 rows

,Order Number,CustomerID,Sales Channel,Order Date,Order Shipping Country,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Order Status,Order Shipping Service,Order Discount Code
0,#0002427,6218,Gymshark SWE Shopify Store,19/11/2018,Sweden,178.52,0.00,178.52,103.40,Closed,Standard (3-5 Working Days),NaN
1,#0000628,6222,Gymshark UK Shopify Store,12/10/2016,United Kingdom,32.00,6.00,38.00,31.67,fulfilled,Express Next Day,NaN
2,#0009013,6222,Gymshark UK Shopify Store,04/01/2017,United Kingdom,16.00,4.00,20.00,16.67,fulfilled,Standard (3-5 Working Days),NaN
3,#0012990,6222,Gymshark UK Shopify Store,26/05/2017,United Kingdom,32.00,6.00,38.00,31.67,fulfilled,Next Working Day,NaN
4,#0013994,6222,Gymshark UK Shopify Store,09/11/2018,United Kingdom,40.00,0.00,40.00,33.33,Billed,Standard (2 Working Days),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9891,#0011495,2395,Gymshark EU Shopify Store,01/10/2016,Belgium,71.36,0.00,71.36,58.97,fulfilled,Standard (3-5 Working Days),NaN
9892,#0010547,2395,Gymshark EU Shopify Store,31/10/2016,Belgium,108.82,0.00,108.82,89.93,fulfilled,Standard (3-5 Working Days),NaN
9893,#0010281,2395,Gymshark EU Shopify Store,17/01/2019,Belgium,39.93,4.43,44.36,36.66,Billed,Standard (2-3 Working Days),NaN
9894,#0012568,2395,Gymshark EU Shopify Store,20/02/2019,Belgium,78.19,0.00,78.19,64.62,Billed,Standard (2-3 Working Days),NaN


## Initial observations from the data:

- ### Don't always have to pay for shipping, could be spending over certain amount
- ### Despite the different countries, GBP is used as metric
- ### The columns totals do not add up for many of the records here, as the 'Order Net Total' is sometimes rounded up or down E.g. record 2, 'Order net total' 16.67, rounding down becoming 'Order items gross' 16.00, adding the shipping to become 20.00, or record 3, 'Order net total' 31.67 rounding up in 'Order items gross' to 32.00, adding shipping to become 38.00
- ### However, this could be due to tax (Order Item table). 

In [156]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9896 entries, 0 to 9895
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Order Number              9896 non-null   object 
 1   CustomerID                9896 non-null   int64  
 2   Sales Channel             9894 non-null   object 
 3   Order Date                9896 non-null   object 
 4   Order Shipping Country    9882 non-null   object 
 5   Order Items Gross         9896 non-null   float64
 6   Order Shipping Gross GBP  9896 non-null   float64
 7   Order Gross Total GBP     9896 non-null   float64
 8   Order Net Total GBP       9896 non-null   float64
 9   Order Status              9896 non-null   object 
 10  Order Shipping Service    9891 non-null   object 
 11  Order Discount Code       1902 non-null   object 
dtypes: float64(4), int64(1), object(7)
memory usage: 927.9+ KB


###### In the Order table, there is:

  - ### 2 missing 'Sales Channel' entries 
  - ### 14 missing 'Order Shipping Country' 
  - ### 5 missing 'Order Shipping Service'
  - ### 7994 missing 'Order Discount Code' 
 
### Likely not much concern for the above, however, the 76 duplicated order numbers, 9 'Order Status' types, and 69 different 'Order Shipping Service' types may be of concern ↓

In [157]:
df_order.describe(include = ['object', 'int'])

,Order Number,Sales Channel,Order Date,Order Shipping Country,Order Status,Order Shipping Service,Order Discount Code
count,9896,9894,9896,9882,9896,9891,1902
unique,9824,14,1353,62,9,69,1233
top,#0004818,Gymshark USA Shopify Store,19/11/2018,United States,Billed,Standard (3-5 Working Days),SAVEFIVE
freq,3,4105,621,3817,6770,2201,95


## Duplicated Order Numbers:

In [158]:
df_order.duplicated(subset='Order Number').value_counts()

False    9824
True       72
dtype: int64

In [159]:
df_order['Order Number'].value_counts()

#0004818    3
#0012211    3
#0011095    3
#0004053    2
#0006000    2
           ..
#0012163    1
#0004740    1
#0010771    1
#0010959    1
#0008933    1
Name: Order Number, Length: 9824, dtype: int64

### ^ Order numbers should not be duplicated, would need further investigation on dates they were duplicated etc.

### Looking at customers who order more than once:

In [160]:
df_order.duplicated(subset='CustomerID', keep=False).value_counts() 

True     6385
False    3511
dtype: int64

### ^ 3511 orders were made by one-time customers, the other orders were collectively made by repeat customers

## Order Status column:

In [161]:
df_order = df_order.rename(columns = {'Order Status' : 'Order_Status'})
df_order.Order_Status.value_counts()

Billed         6770
fulfilled      2344
billed          550
Closed          147
Unfulfilled      39
Cancelled        27
partial           9
unfulfilled       9
restocked         1
Name: Order_Status, dtype: int64

### We can see that there is redundancies between the types of order statuses, e.g. 'Billed'/'billed' and 'Unfulfilled'/'unfulfilled'. Need to be consistent. 

## Order Shipping Service column:

In [162]:
df_order['Order Shipping Service'].value_counts() 

Standard (3-5 Working Days)           2201
Standard (7-14 Working Days)          1958
Standard (4-7 Working Days)           1397
Shipping                               720
Standard (2 Working Days)              711
                                      ... 
Heavy Order International Shipping       1
Express Tracked Large Order              1
EU Standard Mail FREE                    1
Norway Standard Mail FREE                1
Brazil Express Tracked Shipping          1
Name: Order Shipping Service, Length: 69, dtype: int64

In [163]:
df_order['Order Shipping Service'].unique()

array(['Standard (3-5 Working Days)', 'Express Next Day',
       'Next Working Day', 'Standard (2 Working Days)',
       'Standard (7-14 Working Days)', 'Standard (4-7 Working Days)',
       'Standard (2-3 Working Days)', 'Shipping',
       'Express (1-2 Working Days)', 'Express (2-4 Working Days)',
       'Standard (5-10 Working Days)', 'Failed Shipment Mapping',
       'Express (1-3 Working Days)', 'Free shipping - Internal',
       'Standard (3-5 Working Day',
       'Express Tracked Large Order (Republic of Ireland Only)',
       'Royal Mail Tracked 48', 'UK DPD Next Day',
       'Standard (3-6 Working Days)', 'USA Express Tracked Shipping',
       'Royal Mail Italy Tracked and Signed', 'Standard',
       'Norway Standard Mail FREE', 'Express Tracked Australian Shipping',
       'UK DPD Next Working Day',
       'Express Tracked Shipping (Republic of Ireland Only)',
       'Next Working Day (free over £50)', 'Express',
       'Brazil Express Tracked Shipping', nan, 'Norway Standard

### It is likely that these could be condensed into fewer types of shipping

## Looking at values of the orders:

In [164]:
df_order.describe(include=['float'])

,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP
count,9896.000000,9896.000000,9896.000000,9896.000000
mean,57.414705,2.893868,60.499128,55.115453
std,45.485931,3.599457,44.317118,42.088935
min,0.000000,0.000000,0.000000,0.000000
25%,30.187500,0.000000,35.717500,30.985000
50%,47.350000,0.000000,50.905000,46.050000
75%,73.362500,4.460000,74.965000,68.000000
max,1932.650000,26.650000,1932.650000,1932.650000


## ^ From describing the statistics:

- ### Some orders are processed without any purchases, where Min gross and net totals are Zero
- ### Potentially some outliers of extremely high order totals of ~£1.9k
- ### Also see from the max record that shipping cost is not always correctly added

### Investigating the price range of the orders:

In [165]:
print(df_order['Order Gross Total GBP'].value_counts(bins=8, sort = False, dropna = False))

(-1.934, 241.581]       9846
(241.581, 483.162]        47
(483.162, 724.744]         2
(724.744, 966.325]         0
(966.325, 1207.906]        0
(1207.906, 1449.488]       0
(1449.488, 1691.069]       0
(1691.069, 1932.65]        1
Name: Order Gross Total GBP, dtype: int64


### ^ We see the one extremely high purchase is an outlier, potentially a system error, would need investigating

### Also see some purchases are potentially negative:

In [166]:
print((df_order['Order Gross Total GBP'] < 0).value_counts()) # None are negative

False    9896
Name: Order Gross Total GBP, dtype: int64


In [167]:
print((df_order['Order Gross Total GBP'] == 0).value_counts()) # However 250 records are total 0

False    9646
True      250
Name: Order Gross Total GBP, dtype: int64


### ^ The records containing records of total zero can likely be removed

## Order Date investigations

In [168]:
df_order['Order Date'] = pd.to_datetime(df_order['Order Date']) 

print("Most recent order: ", df_order['Order Date'].max())
print("Oldest order: ", df_order['Order Date'].min())

Most recent order:  2019-12-05 00:00:00
Oldest order:  2015-01-02 00:00:00


In [169]:
df_order['Order Date'] = pd.to_datetime(df_order['Order Date']) 
print(df_order['Order Date'].value_counts(bins=8, sort = False, dropna = False))

(2014-12-31 04:50:52.799999999, 2015-08-14 18:00:00]     223
(2015-08-14 18:00:00, 2016-03-26 12:00:00]               378
(2016-03-26 12:00:00, 2016-11-06 06:00:00]               519
(2016-11-06 06:00:00, 2017-06-19]                       1021
(2017-06-19, 2018-01-29 18:00:00]                       1733
(2018-01-29 18:00:00, 2018-09-11 12:00:00]              2162
(2018-09-11 12:00:00, 2019-04-24 06:00:00]              3220
(2019-04-24 06:00:00, 2019-12-05]                        640
Name: Order Date, dtype: int64


### ^ All orders created within reasonable range

***

# Order Item Table

In [170]:
df_order_item = pd.read_csv('.\Order Item.csv')

In [171]:
df_order_item # 31,296 rows

,Order Number,CustomerID,Item Quantity,Item SKU,Item Subtotal (Inc Disc),Item Discount GBP,Item Gross Total GBP,Item Tax Value GBP,Item Net Total GBP
0,#0004761,6257,6,GLLT026-BKM-L,91.08,0.0,91.08,0.00,91.08
1,#0002041,332,4,GFCSTP-BK-S,20.54,0.0,20.54,0.00,20.54
2,#0008264,9129,4,GMBG002-L-BK,64.00,NaN,64.00,10.67,53.33
3,#0008500,7389,4,GLUW005-BK-XS,36.83,0.0,36.83,0.00,36.83
4,#0008585,6257,4,GLST003-BKM-L,115.36,NaN,103.83,0.00,103.83
...,...,...,...,...,...,...,...,...,...
31291,#0014277,8852,0,STANDARD (3-5 WORKING DAYS),0.00,0.0,0.00,0.00,0.00
31292,#0014278,6645,0,STANDARD (3-5 WORKING DAYS),0.00,0.0,0.00,0.00,0.00
31293,#0014279,9916,0,STANDARD (3-5 WORKING DAYS),0.00,0.0,0.00,0.00,0.00
31294,#0014280,6244,0,STANDARD (2-3 WORKING DAYS),0.00,0.0,0.00,0.00,0.00


## First impressions:

- ### The 'Item Gross Total' column does not add up for all of the records, made up of 'Item Net Total GBP' and 'Item Tax Value GBP'

- ### The discount value is added onto the subtotal for the item, rather than being subtracted, e.g. record 1 has a 'Item Gross Total GBP' of 25.11, and an 'Item Discount' of 2.79. The 'Item Subtotal' should be 22.32, not 27.9.

- ### We expect there to be duplicated order numbers and customer numbers, as there may be multiple items per order, and multiple orders over time


## Investigating errors in calulation of:

##### Item Gross Total

In [172]:
df_order_item = df_order_item.rename(columns = {'Item Subtotal (Inc Disc)' : 'Item_Subtotal_Inc_Disc', 'Item Discount GBP' : 'Item_Discount_GBP', 'Item Gross Total GBP' : 'Item_Gross_Total_GBP', 'Item Tax Value GBP' : 'Item_Tax_Value_GBP', 'Item Net Total GBP' :'Item_Net_Total_GBP' })

In [173]:
print((df_order_item.Item_Net_Total_GBP + df_order_item.Item_Tax_Value_GBP
       == df_order_item.Item_Gross_Total_GBP).value_counts(dropna= False))

True     28455
False     2841
dtype: int64


### ^ 2841 incorrect calulations of the Item Gross Total (Item Net Total + Item Tax Value)

###### Subtraction of Discount to Item Subtotal (Inc Disc)

In [174]:
(df_order_item.Item_Discount_GBP > 0.00).value_counts(dropna=True)

False    26721
True      4575
Name: Item_Discount_GBP, dtype: int64

In [175]:
## adding it 
print(((df_order_item.Item_Gross_Total_GBP + df_order_item.Item_Discount_GBP 
        == df_order_item.Item_Subtotal_Inc_Disc)).value_counts(dropna=False))


True     27278
False     4018
dtype: int64


### ^ Of the 4,575 discounted items,  4,018 discounts were incorrectly calculated.

## Overview of the data:

In [176]:
df_order_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31296 entries, 0 to 31295
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Order Number            31296 non-null  object 
 1   CustomerID              31296 non-null  int64  
 2   Item Quantity           31296 non-null  int64  
 3   Item SKU                31296 non-null  object 
 4   Item_Subtotal_Inc_Disc  31296 non-null  float64
 5   Item_Discount_GBP       28309 non-null  float64
 6   Item_Gross_Total_GBP    31296 non-null  float64
 7   Item_Tax_Value_GBP      31296 non-null  float64
 8   Item_Net_Total_GBP      31296 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 2.1+ MB


### ^ All of the records are full in every column, apart from missing entries in 'Item_Discount_GBP'

In [177]:
df_order_item.describe()

,CustomerID,Item Quantity,Item_Subtotal_Inc_Disc,Item_Discount_GBP,Item_Gross_Total_GBP,Item_Tax_Value_GBP,Item_Net_Total_GBP
count,31296.000000,31296.000000,31296.000000,28309.000000,31296.000000,31296.000000,31296.000000
mean,5028.302627,0.750863,19.360660,0.380168,18.954850,1.575663,17.381286
std,2887.177632,0.458903,13.776622,1.072332,13.515765,2.359546,12.369531
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2522.000000,0.000000,7.480000,0.000000,7.377500,0.000000,7.070000
50%,5092.000000,1.000000,19.390000,0.000000,18.950000,0.000000,17.030000
75%,7489.000000,1.000000,29.770000,0.000000,28.790000,3.130000,26.560000
max,10000.000000,6.000000,152.730000,29.860000,152.730000,30.540000,122.190000


In [178]:
df_order_item.describe(include=["object", "int"])

,Order Number,Item SKU
count,31296,31296
unique,9897,7720
top,#0014283,STANDARD (3-5 WORKING DAYS)
freq,55,1612


###  ^ We expect the duplicated order numbers, due to the different items in the orders

### Having observed disparities in the shipping cost and overall cost in the orders table, investigation needed into the shipping cost and whether they are added to the orders.

In [179]:
df_shipping = pd.DataFrame()

In [180]:
df_shipping['Order Shipping Gross GBP'] = df_order['Order Shipping Gross GBP']

In [181]:
df_shipping['Item Quantity'] = df_order_item['Item Quantity']

In [182]:
df_shipping['Item SKU or Shipping'] = df_order_item['Item SKU']

In [183]:
df_shipping # 9896 

,Order Shipping Gross GBP,Item Quantity,Item SKU or Shipping
0,0.00,6,GLLT026-BKM-L
1,6.00,4,GFCSTP-BK-S
2,4.00,4,GMBG002-L-BK
3,6.00,4,GLUW005-BK-XS
4,0.00,4,GLST003-BKM-L
...,...,...,...
9891,0.00,1,GLSB017-PW-S
9892,0.00,1,GLLG042-PW-S
9893,4.43,1,GLLG052-BK-S
9894,0.00,1,GAHW009-CP-OS


In [184]:
df_shipping = df_shipping[df_shipping['Item SKU or Shipping'].str.startswith('G') == False] 
# only keep everything which is not a product
df_shipping

,Order Shipping Gross GBP,Item Quantity,Item SKU or Shipping
116,0.00,2,NBLG001-BK-M
1033,6.00,1,NB2LG001-DKBM-XS
1034,0.00,1,NB2SB001-DKBM-XS
1035,11.31,1,NB2SH001-DKBM-XS
1212,4.36,1,NBCT003-BK-S
1213,0.00,1,NBPO001-BK-S
1743,4.00,1,NB2LG001-DKBM-M
1946,4.00,1,NBLG001-BK-M
2983,0.00,1,NB2AIO001-BKM-S
2984,0.00,1,NB2AIO001-DOM-S


In [185]:
df_shipping = df_shipping[df_shipping['Order Shipping Gross GBP'] > 0]
df_shipping

,Order Shipping Gross GBP,Item Quantity,Item SKU or Shipping
1033,6.00,1,NB2LG001-DKBM-XS
1035,11.31,1,NB2SH001-DKBM-XS
1212,4.36,1,NBCT003-BK-S
1743,4.00,1,NB2LG001-DKBM-M
1946,4.00,1,NBLG001-BK-M
3531,11.48,1,NB2SB001-DOM-S
3575,4.45,1,NB2CT002-DKBM-M
3577,4.00,1,NB2SB001-DKBM-M
3718,7.69,1,NBLG001-BK-XS
3733,5.26,1,NB2SB001-BKM-XS


In [186]:
df_shipping['Item SKU or Shipping'].value_counts()

NB2LG001-DKBM-M     3
NBPO001-BK-S        1
NB2LG001-RCM-S      1
NB2SB001-DKBM-M     1
NBLG001-BK-M        1
NBPO001-BPU-S       1
NBLG001-BK-L        1
NB2SB001-BKM-XS     1
NB2CT002-DKBM-M     1
NBLG001-BK-XS       1
NB2SB001-DOM-S      1
NB2LG001-DKBM-XS    1
SCLT001-BK-M        1
NBPO002-ND-M        1
NBCT003-BK-S        1
NB2CT001-RCM-XS     1
NB2SH001-DKBM-XS    1
NB2SB002-PLP-L      1
NB2LG001-RCM-M      1
SCLT001-LTN-M       1
Name: Item SKU or Shipping, dtype: int64

In [187]:
df_shipping[df_shipping['Item Quantity'] == 0]

,Order Shipping Gross GBP,Item Quantity,Item SKU or Shipping


### ^ Therefore, there are 689 items associated with Shipping costs, that are not 'added' to item quantities when being included in the subtotal and gross costs for orders.

***

## Item Table

In [188]:
df_item = pd.read_csv('.\Item.csv')

In [189]:
df_item # 8,319 rows

,Item SKU,Size,Colour,Gender,Product Category,Product Description
0,GMLS004-WH-XL,Extra Large,White,Male,T-Shirts & Tops,Ark Long Sleeve T-Shirt
1,GLSW001-PHC-S,Small,Peach Coral,Female,Underwear,Cut Out Swimsuit
2,GMST010-AG-M,Medium,Alpine Green,Male,T-Shirts & Tops,Apollo T-Shirt V2
3,GMTK3335-BK-L,Large,Black,Male,Tanks,Ascend Tank
4,GLLT026-PHPM-L,Large,Peach Pink Marl,Female,T-Shirts & Tops,Vital Seamless Long Sleeve Top
...,...,...,...,...,...,...
8315,GLVT008-PL-L,Large,Plum,Female,T-Shirts & Tops,Tempo Vest
8316,GMST046-PGNM-M,Medium,Pastel Green Marl,Male,T-Shirts & Tops,Ghost SS T-Shirt
8317,GMPO006-GL-S,Small,Light Grey,Male,Hoodies,Edge 1/4 Zip Pullover
8318,GMLT1945-SMGM-XL,Extra Large,Smokey Grey Marl,Male,NaN,Apollo LS T-Shirt


In [190]:
df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8320 entries, 0 to 8319
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Item SKU             8320 non-null   object
 1   Size                 8239 non-null   object
 2   Colour               8298 non-null   object
 3   Gender               8300 non-null   object
 4   Product Category     7627 non-null   object
 5   Product Description  8320 non-null   object
dtypes: object(6)
memory usage: 390.1+ KB


## ^ Mostly full dataset, with:
- ### 81 missing size's for items
- ### 22 missing colours
- ### 20 missing genders
- ### 693 missing product category

In [191]:
df_item.describe()

,Item SKU,Size,Colour,Gender,Product Category,Product Description
count,8320,8239,8298,8300,7627,8320
unique,8318,12,525,3,13,946
top,GSB,Medium,Black,Female,T-Shirts & Tops,Apollo SS T-Shirt
freq,3,2521,1453,4380,2619,105


## ^ From the described statistics:

- ### 2 duplicate values
- ### 3 genders, needs investigation
- ### size may need checking


In [192]:
df_item[df_item['Item SKU'].duplicated()]

,Item SKU,Size,Colour,Gender,Product Category,Product Description
3810,GSB,NaN,Black,Accessories,Accessories,Shaker Bottle
4111,GSB,NaN,Pink,Accessories,Accessories,Shaker Bottle


### ^ The SKU 'GSB' is duplicated 3 times, despite being different colour items, so a new SKU code is needed

In [193]:
df_item['Gender'].value_counts()

Female         4380
Male           3806
Accessories     114
Name: Gender, dtype: int64

### ^ We can see that the 'Product Catgegory' of Accessories has been duplicated to some items as a gender, which would need resolving

In [194]:
df_item['Product Category'].value_counts() # no overlap

T-Shirts & Tops    2619
Bottoms             996
Leggings            823
Hoodies             783
Sports Bra          603
Shorts              460
Tanks               400
Jackets             301
Stringers           227
Underwear           200
Base Layers         102
Accessories          91
Socks                22
Name: Product Category, dtype: int64

In [195]:
df_item['Size'].value_counts()

Medium         2521
Small          2251
Large          1677
Extra Small    1017
Extra Large     571
XXL             107
One Size         62
0                17
9-12             13
1                 1
2.2L              1
4-8               1
Name: Size, dtype: int64

### ^ The sizes of '0' and '1' could be altered to fit the category correctly

***

In [196]:
df_customer.head(1)

,CustomerID,Billing City,Billing Country,Gender,Date of Birth,Creation Date,First Order Date,Email Opt In
0,4110,Wetzlar,Germany,Female,1982-04-01,2017-12-07,2018-01-01,NaN


In [197]:
df_order.head(1)

,Order Number,CustomerID,Sales Channel,Order Date,Order Shipping Country,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Order_Status,Order Shipping Service,Order Discount Code
0,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.4,Closed,Standard (3-5 Working Days),NaN


In [198]:
df_order_item.head(1)

,Order Number,CustomerID,Item Quantity,Item SKU,Item_Subtotal_Inc_Disc,Item_Discount_GBP,Item_Gross_Total_GBP,Item_Tax_Value_GBP,Item_Net_Total_GBP
0,#0004761,6257,6,GLLT026-BKM-L,91.08,0.0,91.08,0.0,91.08


In [199]:
df_item.head(1)

,Item SKU,Size,Colour,Gender,Product Category,Product Description
0,GMLS004-WH-XL,Extra Large,White,Male,T-Shirts & Tops,Ark Long Sleeve T-Shirt


***

## Joining the tables

- ### Issues with joining the tables would be the use of duplicate order numbers (72 duplicates in 'Order.csv'). 

- ### Not all of the customers from the 'Customer' are represented in the 'Order' table. Some customer IDs are duplicated in 'Order' due to repeat orders. 

### First, join Customer onto Order using 'CustomerID'

In [200]:
df_order.shape, df_customer.shape

((9896, 12), (10000, 8))

In [201]:
new_df = pd.merge(df_order, df_customer, how='left', left_on ='CustomerID', right_on='CustomerID')
new_df

,Order Number,CustomerID,Sales Channel,Order Date,Order Shipping Country,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Order_Status,Order Shipping Service,Order Discount Code,Billing City,Billing Country,Gender,Date of Birth,Creation Date,First Order Date,Email Opt In
0,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.00,178.52,103.40,Closed,Standard (3-5 Working Days),NaN,Lund,Sweden,NaN,NaT,2018-09-19,2018-11-19,True
1,#0000628,6222,Gymshark UK Shopify Store,2016-12-10,United Kingdom,32.00,6.00,38.00,31.67,fulfilled,Express Next Day,NaN,Workington,United Kingdom,NaN,NaT,2016-10-11,2016-10-12,NaN
2,#0009013,6222,Gymshark UK Shopify Store,2017-04-01,United Kingdom,16.00,4.00,20.00,16.67,fulfilled,Standard (3-5 Working Days),NaN,Workington,United Kingdom,NaN,NaT,2016-10-11,2016-10-12,NaN
3,#0012990,6222,Gymshark UK Shopify Store,2017-05-26,United Kingdom,32.00,6.00,38.00,31.67,fulfilled,Next Working Day,NaN,Workington,United Kingdom,NaN,NaT,2016-10-11,2016-10-12,NaN
4,#0013994,6222,Gymshark UK Shopify Store,2018-09-11,United Kingdom,40.00,0.00,40.00,33.33,Billed,Standard (2 Working Days),NaN,Workington,United Kingdom,NaN,NaT,2016-10-11,2016-10-12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9891,#0011495,2395,Gymshark EU Shopify Store,2016-01-10,Belgium,71.36,0.00,71.36,58.97,fulfilled,Standard (3-5 Working Days),NaN,Habay,Belgium,Male,1994-05-17,2016-08-08,2016-09-10,NaN
9892,#0010547,2395,Gymshark EU Shopify Store,2016-10-31,Belgium,108.82,0.00,108.82,89.93,fulfilled,Standard (3-5 Working Days),NaN,Habay,Belgium,Male,1994-05-17,2016-08-08,2016-09-10,NaN
9893,#0010281,2395,Gymshark EU Shopify Store,2019-01-17,Belgium,39.93,4.43,44.36,36.66,Billed,Standard (2-3 Working Days),NaN,Habay,Belgium,Male,1994-05-17,2016-08-08,2016-09-10,NaN
9894,#0012568,2395,Gymshark EU Shopify Store,2019-02-20,Belgium,78.19,0.00,78.19,64.62,Billed,Standard (2-3 Working Days),NaN,Habay,Belgium,Male,1994-05-17,2016-08-08,2016-09-10,NaN


### SQL:

#### SELECT * FROM df_order
#### JOIN df_customer ON
#### df_order.CustomerID = df_customer.CustomerID ; 

### Then join 'order_item' Table onto the new table using Order Number and Customer ID (due to duplicate Order Numbers)

In [202]:
new_df  = pd.merge(new_df, df_order_item, how='left', left_on = ['Order Number', 'CustomerID'], right_on=['Order Number', 'CustomerID'])

In [203]:
new_df

,Order Number,CustomerID,Sales Channel,Order Date,Order Shipping Country,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Order_Status,...,Creation Date,First Order Date,Email Opt In,Item Quantity,Item SKU,Item_Subtotal_Inc_Disc,Item_Discount_GBP,Item_Gross_Total_GBP,Item_Tax_Value_GBP,Item_Net_Total_GBP
0,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMPO020-BKM-S,21.61,0.0,21.61,4.32,17.29
1,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMHD030-GLM-S,33.29,0.0,33.29,6.66,26.63
2,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMST052-LGM-M,9.07,0.0,9.07,1.81,7.26
3,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMST054-BKM-M,15.12,0.0,15.12,3.02,12.10
4,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMST059-BKM-M,10.37,0.0,10.37,2.07,8.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31243,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1830-BKM-L,43.18,0.0,43.18,7.20,35.98
31244,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1831-BK-L,47.50,0.0,47.50,7.92,39.58
31245,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1833-BK-L,47.50,0.0,47.50,7.92,39.58
31246,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1833-WKH-L,47.50,0.0,47.50,7.92,39.58


### ^ This adds all of the items from each order using the order number and customer ID's, increasing the table from 9,896 rows to 31,248.  

### --> All column names should have been changed to underscores for ease of use

### SQL:

#### SELECT * FROM new_df
#### JOIN df_order_item ON
#### new_df.Order_Number = df_order_item.Order_Number
#### AND new_df.CustomerID = df_order_item.CustomerID; 

## Finally, add the product items to the table for the all of the order data, by joining 'Item' Table on 'Item SKU'  (one duplicate SKU, would need to be unique before the final join).

In [204]:
new_2 = pd.merge(new_df, df_item, how='left', left_on='Item SKU', right_on='Item SKU')
new_2

,Order Number,CustomerID,Sales Channel,Order Date,Order Shipping Country,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Order_Status,...,Item_Subtotal_Inc_Disc,Item_Discount_GBP,Item_Gross_Total_GBP,Item_Tax_Value_GBP,Item_Net_Total_GBP,Size,Colour,Gender_y,Product Category,Product Description
0,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,21.61,0.0,21.61,4.32,17.29,Small,Black Marl,Male,Hoodies,Jacquard Pullover
1,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,33.29,0.0,33.29,6.66,26.63,Small,Light Grey,Male,Hoodies,Eaze Zip Hoodie
2,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,9.07,0.0,9.07,1.81,7.26,Medium,Light Grey Marl,Male,T-Shirts & Tops,Heather SS T-Shirt
3,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,15.12,0.0,15.12,3.02,12.10,Medium,Black Marl,Male,T-Shirts & Tops,Statement SS T-Shirt
4,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,10.37,0.0,10.37,2.07,8.30,Medium,Black Marl,Male,T-Shirts & Tops,Breathe SS T-Shirt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31261,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,43.18,0.0,43.18,7.20,35.98,Large,Black Marl,Female,Bottoms,Vital Seamless Leggings
31262,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,47.50,0.0,47.50,7.92,39.58,Large,Black,Female,Bottoms,Geo-Mesh Leggings
31263,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,47.50,0.0,47.50,7.92,39.58,Large,Black,Female,Bottoms,True Texture Leggings
31264,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,47.50,0.0,47.50,7.92,39.58,Large,Washed Khaki,Female,Bottoms,True Texture Leggings


### SQL:

#### SELECT * FROM new_df
#### JOIN df_item ON
#### new_df.Item_SKU = df_item.Item_SKU ;

### Shape of the final table of all orders:

In [205]:
new_df.shape ## 31,248 rows, 26 columns from all 4 tables

(31248, 26)

***

## Some aggregations:

### E.g. grouping by order number, to see the details of all items for a customer's order

In [206]:
new_df.groupby(['Order Number', 'CustomerID']).head() 
# Each item in the order is shown, as grouped by the order number and customer ID

,Order Number,CustomerID,Sales Channel,Order Date,Order Shipping Country,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Order_Status,...,Creation Date,First Order Date,Email Opt In,Item Quantity,Item SKU,Item_Subtotal_Inc_Disc,Item_Discount_GBP,Item_Gross_Total_GBP,Item_Tax_Value_GBP,Item_Net_Total_GBP
0,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMPO020-BKM-S,21.61,0.0,21.61,4.32,17.29
1,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMHD030-GLM-S,33.29,0.0,33.29,6.66,26.63
2,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMST052-LGM-M,9.07,0.0,9.07,1.81,7.26
3,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMST054-BKM-M,15.12,0.0,15.12,3.02,12.10
4,#0002427,6218,Gymshark SWE Shopify Store,2018-11-19,Sweden,178.52,0.0,178.52,103.40,Closed,...,2018-09-19,2018-11-19,True,1.0,GMST059-BKM-M,10.37,0.0,10.37,2.07,8.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31243,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1830-BKM-L,43.18,0.0,43.18,7.20,35.98
31244,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1831-BK-L,47.50,0.0,47.50,7.92,39.58
31245,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1833-BK-L,47.50,0.0,47.50,7.92,39.58
31246,#0004144,2398,Gymshark EU Shopify Store,2019-04-15,Austria,185.67,0.0,185.67,154.72,Billed,...,2019-04-14,2019-04-15,NaN,1.0,GLLG1833-WKH-L,47.50,0.0,47.50,7.92,39.58


### SQL:

#### SELECT * FROM new_df
#### GROUP BY Order_Number, CustomerID ;

### E.g. can perform aggregates to find out the total number of items per order

In [207]:
new_df.groupby(['Order Number', 'CustomerID']).sum('Item Quantity')

,,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Item Quantity,Item_Subtotal_Inc_Disc,Item_Discount_GBP,Item_Gross_Total_GBP,Item_Tax_Value_GBP,Item_Net_Total_GBP
Order Number,CustomerID,,,,,,,,,,
#0000001,3106,119.70,11.97,131.67,105.33,1.0,43.89,0.00,43.89,8.78,35.11
#0000002,6129,857.40,0.00,857.40,857.40,5.0,142.90,0.00,142.90,0.00,142.90
#0000003,2922,20.56,3.85,24.41,24.41,1.0,20.56,0.00,20.56,0.00,20.56
#0000004,9177,773.22,0.00,773.22,639.06,5.0,128.88,0.00,128.88,22.37,106.51
#0000005,638,65.22,23.31,88.53,88.53,2.0,29.51,0.00,29.51,0.00,29.51
...,...,...,...,...,...,...,...,...,...,...,...
#0014279,9916,391.55,0.00,391.55,391.55,4.0,78.31,0.00,78.31,0.00,78.31
#0014280,6244,1738.50,0.00,1738.50,1448.70,9.0,193.17,19.31,173.86,29.00,144.87
#0014281,433,45.60,15.00,60.60,60.60,1.0,30.30,0.00,30.30,0.00,30.30


### SQL:

#### SELECT Order_Number, CustomerID, Item_SKU, 
#### Product_Description, SUM(Item_Quantity)
#### FROM new_df
#### GROUP BY Order_Number, CustomerID ;

### e.g. the total number sold of each item

In [208]:
by_item_sales = (new_df.groupby('Item SKU', sort=False)).sum('Item Quantity')
by_item_sales

,CustomerID,Order Items Gross,Order Shipping Gross GBP,Order Gross Total GBP,Order Net Total GBP,Item Quantity,Item_Subtotal_Inc_Disc,Item_Discount_GBP,Item_Gross_Total_GBP,Item_Tax_Value_GBP,Item_Net_Total_GBP
Item SKU,,,,,,,,,,,
GMPO020-BKM-S,14504,209.62,4.32,213.94,132.92,2.0,56.17,3.46,52.71,9.50,43.21
GMHD030-GLM-S,8755,257.96,11.33,269.29,194.17,2.0,78.62,0.00,78.62,6.66,71.96
GMST052-LGM-M,6218,178.52,0.00,178.52,103.40,1.0,9.07,0.00,9.07,1.81,7.26
GMST054-BKM-M,65352,1320.65,24.83,1345.48,1183.92,13.0,192.61,0.78,191.82,15.54,176.28
GMST059-BKM-M,26966,403.33,3.90,407.23,332.11,4.0,52.16,0.82,51.34,2.07,49.27
...,...,...,...,...,...,...,...,...,...,...,...
GMLT002-AGM-L,2395,108.82,0.00,108.82,89.93,1.0,35.68,0.00,35.68,6.19,29.49
GMLT002-CRMM-L,2395,108.82,0.00,108.82,89.93,1.0,35.68,0.00,35.68,6.19,29.49
GMBT1954-WMBG-L,2395,78.19,0.00,78.19,64.62,1.0,39.10,0.00,39.10,6.79,32.31


### SQL:

#### SELECT Item_SKU, SUM(Item_Quantity)
#### FROM new_df
#### GROUP BY Item_SKU ;

### Other aggregates can be calculated from the final table of customer orders, such as the average order gross total, mean number of items per order etc. 

### Additional groupings for the data would involve grouping by the Product category, Shipping country etc. for some basic insights. 